In [1]:
%run init.ipynb

matchzoo version 1.0
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [2]:
preprocessor = mz.models.MatchSRNN.get_default_preprocessor()

In [3]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 9254.14it/s] 
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 5728.49it/s]
Building Vocabulary from a datapack.: 100%|██████████| 418401/418401 [00:00<00:00, 2042684.61it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 8166.90it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 5000.14it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 890080.21it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 9729.90it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00<0

In [4]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7fe0300bc470>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7fdf95c41a20>,
 'vocab_size': 30058,
 'embedding_input_dim': 30058}

In [5]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=100)
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [6]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed
)

In [7]:
padding_callback = mz.models.ArcII.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    batch_size=20,
    stage='train',
    resample=True,
    sort=False,
    callback=padding_callback
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    batch_size=20,
    stage='dev',
    callback=padding_callback
)

In [8]:
model = mz.models.MatchSRNN()

model.params['task'] = ranking_task
model.params['embedding'] = embedding_matrix
model.params['channels'] = 4
model.params['units'] = 10
model.params['dropout'] = 0.2
model.params['direction'] = 'lt'

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

MatchSRNN(
  (embedding): Embedding(30058, 100)
  (dropout): Dropout(p=0.2, inplace=False)
  (matching_tensor_network): MatchingTensor()
  (spatial_gru_network): SpatialGRU(
    (_activation): Tanh()
    (_recurrent_activation): Sigmoid()
  )
  (out): Linear(in_features=10, out_features=1, bias=True)
)
Trainable params:  3048611


In [ ]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [ ]:
trainer.run()

[Iter-102 Loss-0.999]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4394 - normalized_discounted_cumulative_gain@5(0.0): 0.5073 - mean_average_precision(0.0): 0.4725



[Iter-204 Loss-0.909]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4819 - normalized_discounted_cumulative_gain@5(0.0): 0.5523 - mean_average_precision(0.0): 0.5068



[Iter-306 Loss-0.645]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5388 - normalized_discounted_cumulative_gain@5(0.0): 0.6054 - mean_average_precision(0.0): 0.5656



[Iter-408 Loss-0.455]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5272 - normalized_discounted_cumulative_gain@5(0.0): 0.5951 - mean_average_precision(0.0): 0.5551



[Iter-510 Loss-0.320]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.549 - normalized_discounted_cumulative_gain@5(0.0): 0.6101 - mean_average_precision(0.0): 0.5689



[Iter-612 Loss-0.212]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5406 - normalized_discounted_cumulative_gain@5(0.0): 0.6201 - mean_average_precision(0.0): 0.5764



[Iter-714 Loss-0.132]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5232 - normalized_discounted_cumulative_gain@5(0.0): 0.5968 - mean_average_precision(0.0): 0.5499



[Iter-816 Loss-0.102]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5418 - normalized_discounted_cumulative_gain@5(0.0): 0.6076 - mean_average_precision(0.0): 0.569



[Iter-918 Loss-0.071]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5418 - normalized_discounted_cumulative_gain@5(0.0): 0.6056 - mean_average_precision(0.0): 0.5647



[Iter-1020 Loss-0.063]:
